In [1]:
from sklearn.inspection import permutation_importance
import time 
import tensorflow as tf
from tensorflow import keras

from Script import *
df_train = pd.read_csv("Data/train.csv")
df_test  = pd.read_csv("Data/test.csv")

df_train = df_train.drop("i", axis=1)
y = df_train.pop("y")
df_train.insert(54,"y", y)
df_test = df_test.drop(["y", "i"], axis = 1)


df_train, df_test = preprocess_data(df_train, df_test, verbose=False)

col_c = list(df_train.filter(like='c', axis=1).columns)
col_o = list(df_train.filter(like='o', axis=1).columns)

df_train = df_train.drop(col_c+col_o, axis=1)
df_test = df_test.drop(col_c+col_o, axis=1)

X = df_train.iloc[:, 0:-1]
y = df_train.iloc[:, -1]

Xn = keras.utils.normalize(X)

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

X_train, X_test, y_train, y_test = train_test_split(Xn,
                                                   y, test_size=0.2,
                                                   random_state = 1)




In [3]:
y_train

/home/antonio/miniconda3/envs/TFgpu/lib/python3.9/site-packages/pandas/io/formats/format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


4427    0.0
3419    0.0
1576    0.0
3483    1.0
1077    1.0
       ... 
905     0.0
5192    1.0
3980    1.0
235     1.0
5157    1.0
Name: y, Length: 4665, dtype: category
Categories (2, float64): [0.0, 1.0]

In [4]:
units_per_layer = [50, 50, 50, 50, 50, 50]
input_s = X_train.shape[1]
output_s = 1
activation_ = 'relu'



model = Sequential()
depth = len(units_per_layer)
model.add(Dense(units_per_layer[0], activation=activation_, input_shape=(input_s,)))
for i in range(1, depth):
    model.add(Dense(units_per_layer[i], activation=activation_))
    model.add(Dropout(rate = 0.1))
model.add(Dense(output_s, activation = 'sigmoid'))   

2021-11-24 10:02:19.458814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-24 10:02:19.459133: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64
2021-11-24 10:02:19.459200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64
2021-11-24 10:02:19.459283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRAR

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                2050      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0

In [5]:
batch = 512
verbose = True
epochs = 150
checkpoint_filepath = "Temp/"

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50, verbose=True)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                mode='min',
                                                save_best_only=True,
                                                verbose=False)

model.fit(X_train, y_train, epochs=epochs, batch_size=batch, verbose=verbose, validation_split=0.2, callbacks = [early_stopping, model_checkpoint])

#predictions = model.predict(X_test)

/home/antonio/miniconda3/envs/TFgpu/lib/python3.9/site-packages/pandas/io/formats/format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
2021-11-24 10:02:22.993561: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150
8/8 [==============================] - 1s 26ms/step - loss: 0.6806 - accuracy: 0.6334 - val_loss: 0.6624 - val_accuracy: 0.6549
Epoch 2/150
8/8 [==============================] - 0s 5ms/step - loss: 0.6532 - accuracy: 0.6498 - val_loss: 0.6319 - val_accuracy: 0.6549


/home/antonio/miniconda3/envs/TFgpu/lib/python3.9/site-packages/pandas/io/formats/format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


Epoch 3/150
8/8 [==============================] - 0s 6ms/step - loss: 0.6403 - accuracy: 0.6498 - val_loss: 0.6265 - val_accuracy: 0.6549
Epoch 4/150
8/8 [==============================] - 0s 6ms/step - loss: 0.6381 - accuracy: 0.6525 - val_loss: 0.6267 - val_accuracy: 0.6967
Epoch 5/150
8/8 [==============================] - 0s 6ms/step - loss: 0.6336 - accuracy: 0.6750 - val_loss: 0.6206 - val_accuracy: 0.7031
Epoch 6/150
8/8 [==============================] - 0s 5ms/step - loss: 0.6279 - accuracy: 0.6870 - val_loss: 0.6124 - val_accuracy: 0.6999
Epoch 7/150
8/8 [==============================] - 0s 5ms/step - loss: 0.6231 - accuracy: 0.6886 - val_loss: 0.6040 - val_accuracy: 0.6988
Epoch 8/150
8/8 [==============================] - 0s 6ms/step - loss: 0.6167 - accuracy: 0.6881 - val_loss: 0.5999 - val_accuracy: 0.6977
Epoch 9/150
8/8 [==============================] - 0s 5ms/step - loss: 0.6088 - accuracy: 0.6881 - val_loss: 0.5999 - val_accuracy: 0.6988
Epoch 10/150
8/8 [=========

In [6]:
sum(np.round(model.predict(X_test)) == np.array(y_test).reshape(-1, 1))/len(y_test)

array([0.6529563])

In [7]:
np.round(model.predict(X_test)).shape
np.array(y_test).shape

(1167,)

In [8]:
pred_nn = np.round(model.predict(keras.utils.normalize(df_test)))

In [34]:
# Script.save_predictions(pred_test, "try_keras")

In [29]:
def maj_vote(arr):
    counts = np.bincount(arr)
    re = np.argmax(counts)
    return  re

def flatten(l):
    for i in l:
        if isinstance(i,list):
            yield from flatten(i)
        else:
            yield i

def fix_shape(preds):
    pred_list_aux = []
    for  l in preds:
        pred_list_aux.append(np.array([int(x) for x in flatten(l)]).reshape(1, -1))
    arrpred = np.array(pred_list_aux).transpose()
    return arrpred

def vote_given_preds(preds):
    arrpred = fix_shape(preds)
    return np.apply_along_axis(maj_vote, axis=1, arr=arrpred)  
    

In [30]:
a = [[1], [0], [1], [0]]
b = [1, 0, 1, 1]
c = [0, 0, 1, 0]
preds = [a, b, c]






vote_given_preds(preds)

# vote_given_preds(preds)

# for i, col in enumerate(preds):
#     print(col)

array([[1, 1, 0],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 0]])

In [20]:
from sklearn.inspection import permutation_importance
import time 

from Script import *





from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from sklearn.ensemble import GradientBoostingClassifier



import joblib 
# rf_grid_search = joblib.load("Temp/rf_grid_search.joblib")

# clf1 = rf_grid_search.best_estimator_

clf1 = joblib.load("Temp/rf_best_light.joblib")



clf2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)

# print("Fitting Classifiers")
# eclf1 = VotingClassifier(estimators=[
#         ('rf', clf1), ('xgb', clf2)], voting='hard')






# print("Hard Voting ")
# hard_scores = cross_val_score(eclf1, X, y, cv=10)

# print("aux stop")


# import joblib
# name_of_model = "rf_best+xgboost_correct_types"

# joblib.dump(hard_scores, "Temp/"+name_of_model+"_hard_scores"+".joblib")


# print("hard voting", hard_scores.mean())


# print("aux stop")

# # joblib.dump(eclf1, "Temp/"+name_of_model+".joblib")

# eclf1.fit(X, y)

# preds_hard = eclf1.predict(df_test)
# save_predictions(preds_hard, "Predictions/"+name_of_model+"_hard")







In [21]:
clf1.fit(X, y)
clf2.fit(X, y)
pred_rf = clf1.predict(df_test)
pred_xg = clf2.predict(df_test)







In [23]:
pred_list = [pred_rf, pred_xg, pred_nn]

In [24]:
pred_list

[array([1., 0., 0., ..., 1., 0., 1.]),
 array([1., 0., 0., ..., 1., 0., 0.]),
 array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]], dtype=float32)]

In [31]:
vote_given_preds(pred_list)
vote_preds = np.apply_along_axis(maj_vote, axis=1, arr=vote_given_preds(pred_list))  



In [32]:
save_predictions(vote_preds, "Temp/bestrf+bestxgb+keras")